In [50]:
import sys
sys.path.append('../')
sys.path.append('../../')

from DataProc.DataProcessor import DataProcessor
from Datasets.BaseballDataset import BaseballDataset
from BaselineModel.BaselineModel import BaselineModel
from TransformerModel.TransformerModelRedisual import *

import torch
import torch.nn as nn
import torch.optim as optim
import math
import torch.nn.functional as F
from torch.utils.data import DataLoader
import json
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler

In [51]:
data_config_path = "../../data/configv2.json"
full_data_path = "../../data/full_cleaned_zone_93.csv"
sequence_length = 200
full_data = pd.read_csv(full_data_path)


In [52]:
scaler_path = "../../data/full_scalers_zone_93.pkl"

In [53]:
#only use 2024 data for now
data_2024 = full_data[full_data['game_date'] > '2024-01-01']

#player names in 2024
names = data_2024['batter_name'].unique()

In [54]:
m_path = "../med_data_zone_experiment/h6_e6_h72_d0_lp0.5_lr1e-05_ep50/transformer_model.pth"
c_path = "../med_data_zone_experiment/h6_e6_h72_d0_lp0.5_lr1e-05_ep50/model_config.json"

transformer_model = TransformerHelper(m_path,c_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [55]:
from sklearn.metrics import mean_absolute_error
import warnings



# I expect to see RuntimeWarnings in this block
with warnings.catch_warnings():
    warnings.simplefilter("ignore", category=RuntimeWarning)

    verbose = False
    cont_errors = []

    events_errors = []
    events_precisions = []

    loc_errors = []
    loc_precisions = []

    player_names = []
    data_sizes = []

    all_event_preds = []
    all_loc_preds = []

    all_event_true = []
    all_loc_true = []



    for name in names:
        # Create dataset for this player
        player_data = data_2024[data_2024['batter_name'] == name].reset_index(drop=True)

        print(name,len(player_data))

        if len(player_data) < sequence_length:
            continue

        player_dataset = BaseballDataset(player_data, data_config_path, sequence_length)

        if len(player_dataset) < 1:
            continue

        player_names.append(name)
        data_sizes.append(len(player_dataset))

        #print(name,len(player_dataset))

        # Get preds and true using this dataset
        player_preds, player_true = transformer_model.make_preds(player_dataset, scaler_path, device, 2000, scale=False)

        #print(player_preds.columns)

        # First 4 columns are continuous preds/labels
        cont_preds = player_preds.iloc[:, 0:4]
        cont_true = player_true.iloc[:, 0:4]

        # Next 10 columns are probabilities for the events categorical feature
        events_preds = player_preds.iloc[:, 4:14]
        events_true = player_true.iloc[:, 4:14]



        # Last 10 columns are probabilities for the hit_location categorical feature
        loc_preds = player_preds.iloc[:, 14:]
        loc_true = player_true.iloc[:, 14:]

        # Compute errors for continuous predictions (using MAE or other metrics)
        cont_error = mean_absolute_error(cont_true, cont_preds)
        cont_errors.append(cont_error)

        if verbose:
            print(f"Continuous error for {name}: {cont_error}")



        # Summing the probability distributions for events and hit_location
        pred_events_sum = events_preds.sum(axis=0)
        true_events_sum = events_true.sum(axis=0)


        all_event_preds.append(pred_events_sum)
        all_event_true.append(true_events_sum)

        pred_loc_sum = loc_preds.sum(axis=0)
        true_loc_sum = loc_true.sum(axis=0)

        all_loc_preds.append(pred_loc_sum)
        all_loc_true.append(true_loc_sum)


        events_sum_abs_errors = np.abs(pred_events_sum - true_events_sum).values
        loc_sum_abs_errors = np.abs(pred_loc_sum - true_loc_sum).values

        if verbose:
            print(f"Sum of absolute errors for Events for {name}: {events_sum_abs_errors}")
            print(f"Sum of absolute errors for Hit Location for {name}: {loc_sum_abs_errors}")

        events_errors.append(events_sum_abs_errors)
        loc_errors.append(loc_sum_abs_errors)


        top_k = 4

        # Get top-k predictions for each sample, returning the numeric index instead of column names
        events_top_k_preds = events_preds.apply(lambda x: x.nlargest(top_k).index.map(lambda name: events_preds.columns.get_loc(name)), axis=1)
        loc_top_k_preds = loc_preds.apply(lambda x: x.nlargest(top_k).index.map(lambda name: loc_preds.columns.get_loc(name)), axis=1)

        # Now the predictions are stored as the numeric indices corresponding to the classes

        # Compute precision for each class
        events_class_precisions = []
        loc_class_precisions = []

        # For each class in events and hit_location, calculate top-k precision
        for class_idx in range(10):
            # For events precision
            true_class_mask = events_true.iloc[:, class_idx] == 1  # Find where this class is the true class
            true_class_indices = events_true.index[true_class_mask]
            
            # Check if this class is in the top-k predictions when it's the true class
            event_precision = np.mean([1 if class_idx in events_top_k_preds.iloc[i] else 0 for i in true_class_indices])
            events_class_precisions.append(event_precision)

            # For hit location precision
            true_class_mask = loc_true.iloc[:, class_idx] == 1  # Find where this class is the true class
            true_class_indices = loc_true.index[true_class_mask]

            # Check if this class is in the top-k predictions when it's the true class
            loc_precision = np.mean([1 if class_idx in loc_top_k_preds.iloc[i] else 0 for i in true_class_indices])
            loc_class_precisions.append(loc_precision)

        if verbose:
            print(f"Class-wise Events Precision for {name}: {events_class_precisions}")
            print(f"Class-wise Hit Location Precision for {name}: {loc_class_precisions}")

        events_precisions.append(events_class_precisions)
        loc_precisions.append(loc_class_precisions)





    





zack gelof 1004
tyler nevin 709
kyle mccann 395
ketel marte 1365
corbin carroll 1376
geraldo perdomo 357
josé herrera 47
eugenio suárez 1368
tyler soderstrom 520
brent rooker 1213
miguel andújar 518
jake mccarthy 844
lourdes gurriel 1298
christian walker 1575
blaze alexander 701
randal grichuk 501
jj bleday 1349
aledmys díaz 101
daz cameron 287
joc pederson 1039
lawrence butler 677
tucker barnhart 403
zack short 391
orlando arcia 1209
sean murphy 357
adam duvall 797
austin riley 1141
connor joe 1157
jack suwinski 955
jared triolo 1047
matt olson 1521
marcell ozuna 1438
ozzie albies 1265
jarred kelenic 955
joey bart 313
nick gonzales 678
rowdy tellez 947
travis d'arnaud 693
edward olivares 784
oneil cruz 1293
bryan reynolds 1513
luke williams 38
andrew mccutchen 1321
colton cowser 1113
heston kjerstad 176
jorge mateo 709
ryan o'hearn 942
austin hays 451
robbie grossman 680
josh smith 1145
marcus semien 1353
davis wendzel 170
jordan westburg 1232
anthony santander 1330
adley rutschman 14

In [56]:
event_class_names = player_dataset.categorical_label_names[0]
loc_class_names = player_dataset.categorical_label_names[1]



# Summary Statistics for Class-wise Errors and Precisions
average_events_class_errors = np.nanmean(events_errors, axis=0)
average_loc_class_errors = np.nanmean(loc_errors, axis=0)

average_events_class_precisions = np.nanmean(events_precisions, axis=0)
average_loc_class_precisions = np.nanmean(loc_precisions, axis=0)

# Create DataFrames for the results
events_summary_df = pd.DataFrame({
    'Class': event_class_names,
    'Average Error': average_events_class_errors,
    f'Average Precision (Top K {top_k})': average_events_class_precisions
})

loc_summary_df = pd.DataFrame({
    'Class': loc_class_names,
    'Average Error': average_loc_class_errors,
    f'Average Precision (Top {top_k})': average_loc_class_precisions
})

# Print formatted DataFrames
print("Events Class-wise Summary:")
print(events_summary_df)

print("\nHit Location Class-wise Summary:")
print(loc_summary_df)

Events Class-wise Summary:
                 Class  Average Error  Average Precision (Top K 4)
0             events_B       0.706605                     0.999063
1             events_S       0.853845                     0.998364
2        events_double       2.161299                     0.904734
3     events_field_out       9.117659                     0.999895
4  events_hit_by_pitch       0.599568                     0.993759
5      events_home_run       2.409214                     0.756374
6        events_single       4.380852                     0.999143
7     events_strikeout       8.299831                     0.999236
8        events_triple       0.714591                     0.000000
9          events_walk       1.526218                     0.978815

Hit Location Class-wise Summary:
              Class  Average Error  Average Precision (Top 4)
0  hit_location_0.0       3.807343                   0.993363
1  hit_location_1.0       1.520963                   0.077212
2  hit_location_

In [57]:
total_error = events_summary_df['Average Error'].sum()

print(f'Average num pitches in datasets: {sum(data_sizes) / len(data_sizes)}')
print(f'Total average error: {total_error}')

Average num pitches in datasets: 635.0314606741573
Total average error: 30.76968002319336


In [75]:
pname = "bryce harper"
idx = player_names.index(pname)


# Create a DataFrame for events errors and precisions
events_df = pd.DataFrame({
    "Error": events_errors[idx],
    "Precision": events_precisions[idx]
}, index=event_class_names)


# Output the DataFrames
print(f"Events DataFrame for {pname}:")
print(f"Num pitches: {data_sizes[idx]}")
print(events_df)

print("\nModel Preds")
print(all_event_preds[idx])
print("\nTrue")
print(all_event_true[idx])


Events DataFrame for bryce harper:
Num pitches: 1109
                        Error  Precision
events_B             2.822998   1.000000
events_S             0.264709   1.000000
events_double        1.106873   0.888889
events_field_out     0.392548   1.000000
events_hit_by_pitch  3.561788        NaN
events_home_run      5.290731   0.529412
events_single        5.454277   1.000000
events_strikeout     7.250343   1.000000
events_triple        1.792187        NaN
events_walk          3.447582   0.971429

Model Preds
events_B               402.822998
events_S               426.264709
events_double           16.893127
events_field_out       108.607452
events_hit_by_pitch      3.561788
events_home_run         11.709269
events_single           37.545723
events_strikeout        68.250343
events_triple            1.792187
events_walk             31.552418
dtype: float32

True
events_B               400.0
events_S               426.0
events_double           18.0
events_field_out       109.0
events

In [59]:
#baseline for this comparison will simply be using a players career averages

#pre-2024 data for averages since we are predicting 2024 stats
baseline_data = full_data[full_data['game_date'] < '2024-01-01']

baseline_events_errors = []
baseline_loc_errors = []


for i, player in enumerate(player_names):


    #get pre-2024 data for this player

    player_data = baseline_data[baseline_data['batter_name'] == player]

    
    if len(player_data) == 0:
        continue
    
    event_cols = ['events_B', 'events_S', 'events_double',
       'events_field_out', 'events_hit_by_pitch', 'events_home_run',
       'events_single', 'events_strikeout', 'events_triple', 'events_walk']
    
    player_event_means = player_data[event_cols].sum(axis=0) / len(full_data)

    #multiply by number of pitches seen for 2024 (won't be entire 2024 since the dataset skips first sequence_length (200) pitches)

    baseline_event_preds = player_event_means * data_sizes[i]
    baseline_event_true = all_event_true[i] #true will be the same as before


    baseline_events_sum_abs_errors = np.abs(baseline_event_preds - baseline_event_true).values


    #same for hit_location

    loc_cols = ['hit_location_0.0', 'hit_location_1.0',
       'hit_location_2.0', 'hit_location_3.0', 'hit_location_4.0',
       'hit_location_5.0', 'hit_location_6.0', 'hit_location_7.0',
       'hit_location_8.0', 'hit_location_9.0']
    
    player_loc_means = player_data[loc_cols].sum(axis=0) / len(full_data)

    #multiply by number of pitches seen for 2024 (won't be entire 2024 since the dataset skips first sequence_length (200) pitches)

    baseline_loc_preds = player_loc_means * data_sizes[i]
    baseline_loc_true = all_loc_true[i] #true will be the same as before


    baseline_loc_sum_abs_errors = np.abs(baseline_loc_preds - baseline_loc_true).values



    baseline_events_errors.append(baseline_events_sum_abs_errors)
    baseline_loc_errors.append(baseline_loc_sum_abs_errors)


    


    

    


    

    
    

   
    


In [64]:
# Summary Statistics for Class-wise Errors and Precisions
average_baseline_events_class_errors = np.nanmean(baseline_events_errors, axis=0)
average_baseline_loc_class_errors = np.nanmean(baseline_loc_errors, axis=0)



# Create DataFrames for the results
baseline_events_summary_df = pd.DataFrame({
    'Class': event_class_names,
    'Average Error': average_baseline_events_class_errors
})

baseline_loc_summary_df = pd.DataFrame({
    'Class': loc_class_names,
    'Average Error': average_baseline_loc_class_errors
})

# Print formatted DataFrames
print("Events Class-wise Summary:")
print(baseline_events_summary_df)

print("\nHit Location Class-wise Summary:")
print(baseline_loc_summary_df)

Events Class-wise Summary:
                 Class  Average Error
0             events_B     213.817410
1             events_S     261.685699
2        events_double       7.112899
3     events_field_out      76.577997
4  events_hit_by_pitch       1.866922
5      events_home_run       4.820044
6        events_single      23.379887
7     events_strikeout      36.098786
8        events_triple       0.658866
9          events_walk      12.987322

Hit Location Class-wise Summary:
              Class  Average Error
0  hit_location_0.0     495.293815
1  hit_location_1.0       3.716910
2  hit_location_2.0      37.325584
3  hit_location_3.0       8.209939
4  hit_location_4.0      12.847200
5  hit_location_5.0      12.103291
6  hit_location_6.0      13.828818
7  hit_location_7.0      17.792462
8  hit_location_8.0      19.698064
9  hit_location_9.0      18.188721


In [150]:
#month to month comparison

#includes about ~200 pitches back from end of march so that we are making preds for every pitch in june
judge_2019_july_aug = full_data[(full_data['batter_name'] == 'whit merrifield') & (full_data['game_date'] > '2021-01-1') & (full_data['game_date'] < '2021-10-01')].reset_index(drop=True)
judge_2019 = BaseballDataset(judge_2019_july_aug,data_config_path,sequence_length)

#includes about ~200 pitches back from end of may so that we are making preds for every pitch in june
judge_2022_july_aug = full_data[(full_data['batter_name'] == 'aaron judge') & (full_data['game_date'] > '2021-01-1') & (full_data['game_date'] < '2021-10-01')].reset_index(drop=True)
judge_2022 = BaseballDataset(judge_2022_july_aug,data_config_path,sequence_length)





judge_2019_preds, judge_2019_true = transformer_model.make_preds(judge_2019, scaler_path, device, 2000, scale=True)
judge_2022_preds, judge_2022_true = transformer_model.make_preds(judge_2022, scaler_path, device, 2000, scale=True)









In [151]:
print(f"total pitches: {judge_2019_preds.sum(axis=0)[4:14].sum(axis=0)}\n")

print("Judge 2019 Preds")
print(judge_2019_preds.sum(axis=0)[4:14])
print()
print("Judge April True")
print(judge_2019_true.sum(axis=0)[4:14])



total pitches: 2370.994609117508

Judge 2019 Preds
events_B               792.275696
events_S               933.656738
events_double           34.115002
events_field_out       333.511017
events_hit_by_pitch      6.234545
events_home_run         20.940908
events_single           92.375763
events_strikeout       113.306778
events_triple            2.323657
events_walk             42.254505
dtype: float64

Judge April True
events_B               792.0
events_S               931.0
events_double           38.0
events_field_out       354.0
events_hit_by_pitch      3.0
events_home_run          7.0
events_single          115.0
events_strikeout        93.0
events_triple            3.0
events_walk             35.0
dtype: float64


In [152]:
print(f"total pitches: {judge_2022_preds.sum(axis=0)[4:14].sum(axis=0)}\n")

print("Judge 2022 Preds")
print(judge_2022_preds.sum(axis=0)[4:14])
print()
print("Judge 2022 True")
print(judge_2022_true.sum(axis=0)[4:14])

total pitches: 2511.9958856105804

Judge 2022 Preds
events_B               939.701599
events_S               982.840942
events_double           26.523649
events_field_out       168.049591
events_hit_by_pitch      3.908535
events_home_run         34.076820
events_single           74.423523
events_strikeout       204.654160
events_triple            2.919055
events_walk             74.898010
dtype: float64

Judge 2022 True
events_B               940.0
events_S               979.0
events_double           24.0
events_field_out       223.0
events_hit_by_pitch      3.0
events_home_run         36.0
events_single           86.0
events_strikeout       151.0
events_triple            0.0
events_walk             70.0
dtype: float64


In [157]:
judge_2019_july_aug[judge_2019_july_aug['hit_location_2.0'] == True]

101